## Task 1

In [2]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
from sklearn import datasets

In [5]:
cov = datasets.fetch_covtype()
cov = pd.DataFrame(data=np.c_[cov.data, cov.target], columns=cov.feature_names+['target'])
values, counts = np.unique(cov['target'].values, return_counts=True)
#for i in range(len(cov['target'])):
#    if cov['target'][i] == values[counts.argmax()]:
#        cov['target'][i] = 1
#    else:
#        cov['target'][i] = 0
cov['target'] = cov['target'].apply(lambda x: 1 if x == values[counts.argmax()] else 0)
np.unique(cov['target'].values)

array([0, 1], dtype=int64)

## Task 2

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
x = cov[cov.columns[:-1]]
y = cov['target'].values
logreg = LogisticRegression(solver='saga')
GBC = GradientBoostingClassifier(random_state = 53)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
print(cross_val_score(logreg, x, y, cv=3, scoring='roc_auc'))
print(cross_val_score(GBC, x, y, cv=3, scoring='roc_auc'))

[0.73666162 0.52608569 0.63262289]
[0.66552099 0.7611432  0.74502843]


In [7]:
from sklearn.preprocessing import MinMaxScaler
Xscale = MinMaxScaler().fit(x).transform(x)
print(cross_val_score(logreg, Xscale, y, cv=3, scoring='roc_auc'))
print(cross_val_score(GBC, Xscale, y, cv=3, scoring='roc_auc'))

[0.88580452 0.77300701 0.70304575]
[0.66552349 0.7611432  0.74502581]


In [8]:
print("the results here suggest that logreg benefits from the minmax scaling. This is likely due to the fact that scaling within log regression will eliminate potential bias, but in GBC, a tree based model, it is largely of no use since it is not dependant on scaling")

the results here suggest that logreg benefits from the minmax scaling. This is likely due to the fact that scaling within log regression will eliminate potential bias, but in GBC, a tree based model, it is largely of no use since it is not dependant on scaling


## Task 3

In [9]:
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import StandardScaler
z = StandardScaler().fit_transform(x)
Xscale = GaussianRandomProjection(n_components=5).fit_transform(z)
print(cross_val_score(logreg, Xscale, y, cv=3, scoring='roc_auc'))

Xscale = GaussianRandomProjection(n_components=20).fit_transform(z)
print(cross_val_score(logreg, Xscale, y, cv=3, scoring='roc_auc'))

from sklearn.decomposition import PCA
Xscale = PCA(n_components=5).fit_transform(z)
print(cross_val_score(logreg, Xscale, y, cv=3, scoring='roc_auc'))

[0.36823827 0.51075911 0.42641674]
[0.54301554 0.67007573 0.67260455]
[0.7684583  0.68544368 0.68933439]


In [10]:
print("the results here suggest that both doing PCA or increasing the number of components will improve model accuracy by roughly the same amount. That said, the reason it happens with 20 dimmensions vs 5 is likely due to the fact that more dimmensions suggests that the data was not compressed to smaller dimmensionality, suggesting less loss. However, PCA appears to have done it better than GRP because PCA attempts to keep the highly correlated features and minimizing reconstruction error.")

the results here suggest that both doing PCA or increasing the number of components will improve model accuracy by roughly the same amount. That said, the reason it happens with 20 dimmensions vs 5 is likely due to the fact that more dimmensions suggests that the data was not compressed to smaller dimmensionality, suggesting less loss. However, PCA appears to have done it better than GRP because PCA attempts to keep the highly correlated features and minimizing reconstruction error.


In [11]:
Xscale = PCA(n_components=5).fit_transform(x)
Xscale = StandardScaler().fit_transform(Xscale)
print(cross_val_score(logreg, Xscale, y, cv=3, scoring='roc_auc'))

[0.67842935 0.52639664 0.60736856]


In [12]:
print("we can see here that the scaling after PCA hurts the results of the model when compared to scaling before PCA. This is likely because the scaling is done to unit variance, which is what PCA tries to retain. Without this, PCA likely losses accuracy")

we can see here that the scaling after PCA hurts the results of the model when compared to scaling before PCA. This is likely because the scaling is done to unit variance, which is what PCA tries to retain. Without this, PCA likely losses accuracy
